# Imviz Demonstration Notebook


This notebook demonstrates the Imviz API in the Notebook setting. UI equivalents for these actions, as well as additional documentation about Imviz, can be found here: https://jdaviz.readthedocs.io/en/latest/imviz/

Import modules needed for this notebook.

In [1]:
import os
import warnings
import tempfile

import matplotlib.pyplot as plt
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord, Angle
from astropy.table import Table
from astroquery.mast import Observations
from photutils.aperture import CircularAperture, SkyCircularAperture
from regions import PixCoord, CirclePixelRegion, CircleSkyRegion

from ipygoldenlayout import GoldenLayout
from ipysplitpanes import SplitPanes
import ipyvue

import jdaviz
from jdaviz.app import custom_components
from jdaviz import Imviz

**WARNING**: LOCAL JWST PRD VERSION PRDOPSSOC-062 CANNOT BE CHECKED AGAINST ONLINE VERSION


We also need this to display Matplotlib in the notebook later.

In [ ]:
# %matplotlib inline

In [ ]:
SplitPanes()
GoldenLayout()
for name, path in custom_components.items():
    ipyvue.register_component_from_file(None, name, os.path.join(os.path.dirname(jdaviz.__file__), path))

ipyvue.register_component_from_file('g-viewer-tab', "container.vue", jdaviz.__file__)


We create an Imviz instance and grab the default viewer instance as well.

In [ ]:
imviz = Imviz()
viewer = imviz.default_viewer

Now we download some data, using `astroquery` to retrieve the files from
[MAST](https://masttest.stsci.edu/). By default the downloaded files go to your
temp directory, and thus may eventually be garbage collected (deleted) by your system.
If you would like to have the file permanently, simply uncomment the second line below
and provide the directory path on your system where you would like the file stored.

One other thing to note about retrieving MAST data through astroquery is that it caches
the data by default. It is possible for files to be updated in MAST with more recent calibrations
but remain the same size, in which case your local cached file would not be replaced by the new
version. You can turn off caching by setting `cache=False` in the `download_file` call to
force it to re-download the file if desired.

In this example, we use two of the JWST NIRCam
observations of the Cartwheel Galaxy. These are large files (~500 MB each), so they make take
some time to download, but after the first time running this notebook they will be cached locally.
If you want to look at images from other filters, you can uncomment the lines with those filenames.
If you want an example using smaller files, see the ImvizDitheredExample notebook, which
includes the same workflow as this one, but using two HST ACS files that are much smaller
than the JWST files used here.

After downloading, each file is loaded into Imviz. We ignore some warnings that appear during parsing.

In [ ]:
data_dir = tempfile.gettempdir()
#data_dir = "/Users/username/Data/"

files = ['jw02727-o002_t062_nircam_clear-f090w_i2d.fits',
         #'jw02727-o002_t062_nircam_clear-f150w_i2d.fits',
         #'jw02727-o002_t062_nircam_clear-f200w_i2d.fits',
         'jw02727-o002_t062_nircam_clear-f277w_i2d.fits',
         #'jw02727-o002_t062_nircam_clear-f356w_i2d.fits',
         #'jw02727-o002_t062_nircam_clear-f444w_i2d.fits'
        ]

for fn in files:
    uri = f"mast:JWST/product/{fn}"
    result = Observations.download_file(uri, local_path=f'{data_dir}/{fn}')
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        imviz.load_data(f'{data_dir}/{fn}')

Then, we visualize the data and start off by looking at some of the basic features:

In [ ]:
# imviz.show()

Alternatively, the data and the configuration can be autodetected and loaded simultaneously by calling `jdaviz.open(f'{data_dir}/{fn}')`

The first thing you will probably notice is that the image doesn't take up the entire viewer area. 
If you press the "b" key to blink to the other loaded image, you will see that this image is 
zoomed correctly by default. The odd default zoom on the other is because the images are linked
by pixel when loaded. We can instead link by WCS (world coordinates) so that the images
will be properly aligned:

In [ ]:
# imviz.link_data(link_type='wcs')

Panning and zooming is possible by showing the viewer toolbar and clicking on the '+'-shaped icon, then dragging around in the image and using scrolling to zoom in and out. To change the stretch and colormap, show the **Layer** options accessible through the last icon in the viewer toolbar.

We can also change these programmatically, for example the stretch: